<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Self_Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

class ContrastiveModel(nn.Module):
    def __init__(self, feature_dim):
        super(ContrastiveModel, self).__init__()
        self.encoder = torchvision.models.resnet18(weights=None, num_classes=feature_dim)

    def forward(self, x):
        return self.encoder(x)

def contrastive_loss(out_1, out_2, temperature):
    out = torch.cat([out_1, out_2], dim=0)
    sim_matrix = F.cosine_similarity(out.unsqueeze(1), out.unsqueeze(0), dim=2)
    labels = torch.arange(out.size(0) // 2).repeat(2).to(out.device)
    mask = torch.eye(out.size(0)).to(out.device).bool()
    sim_matrix = sim_matrix[~mask].view(sim_matrix.size(0), -1)
    sim_matrix = sim_matrix / temperature
    loss = F.cross_entropy(sim_matrix, labels)
    return loss

# Example usage
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

model = ContrastiveModel(feature_dim=128)
optimizer = optim.Adam(model.parameters(), lr=0.001)
temperature = 0.07

for epoch in range(10):
    for images, _ in train_loader:
        out_1 = model(images)
        out_2 = model(images)
        loss = contrastive_loss(out_1, out_2, temperature)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item():.4f}')